In [1]:
import tensorflow as tf

class FusedModel(tf.Module):
  def __init__(self):
    self.color_classifier = tf.saved_model.load('/nfs/general/shared/tf_models/color_classifier')
    self.orientation_classifier=tf.saved_model.load('/nfs/general/shared/tf_models/orientation')

  @tf.function(input_signature=[tf.TensorSpec(shape=[480,640,3], dtype=tf.uint8)])
  def compute(self, image):
    image_cropped_for_color = tf.image.convert_image_dtype(tf.image.crop_to_bounding_box(image, 220, 260, 40, 80), tf.float32)[tf.newaxis,:,:,:]
    image_for_position_detection = tf.image.convert_image_dtype(tf.image.rgb_to_grayscale(tf.image.crop_to_bounding_box(image, 220,70,100,500)), tf.float32)[tf.newaxis,:,:,:]
    orientation=self.orientation_classifier(image_for_position_detection)[0,:]
    return tf.math.softmax(self.color_classifier(image_cropped_for_color))[0,:], orientation


model = FusedModel()
tf.saved_model.save(model, '/tmp/adder')

converter = tf.lite.TFLiteConverter.from_saved_model('/tmp/adder') 

tflite_model = converter.convert()
open('/nfs/general/shared/tflite/fused_model.tflite', 'wb').write(tflite_model)



image = tf.io.decode_jpeg(tf.io.read_file('/nfs/general/shared/KeyD/1650406299896.jpg'))

model.compute(image)

2022-04-19 21:01:41.231348: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 21:01:41.234753: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 21:01:41.235034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 21:01:41.235408: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

INFO:tensorflow:Assets written to: /tmp/adder/assets


2022-04-19 21:01:42.842334: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-19 21:01:42.842353: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-19 21:01:42.842905: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/adder
2022-04-19 21:01:42.846058: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-19 21:01:42.846068: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/adder
2022-04-19 21:01:42.854202: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-19 21:01:42.951369: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/adder
2022-04-19 21:01:42.984820: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 141916 microseconds.
2022

Estimated count of arithmetic ops: 44.671 M  ops, equivalently 22.336 M  MACs


2022-04-19 21:01:43.433305: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2022-04-19 21:01:43.685219: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.9750633 , 0.02493672], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.00694999,  0.9355161 ], dtype=float32)>)

In [1]:
!cat /nfs/general/shared/KeyD/1650406299896.jpg.classification

0.681526	0.662728	0.00190051	0.9981


In [2]:
import tensorflow as tf

class FusedModel(tf.Module):
  def __init__(self):
    self.color_classifier = tf.saved_model.load('/nfs/general/shared/tf_models/color_classifier')
    self.orientation_classifier=tf.saved_model.load('/nfs/general/shared/tf_models/orientation')

  @tf.function(input_signature=[tf.TensorSpec(shape=[480,640,3], dtype=tf.uint8)])
  def compute(self, image):
    image_cropped_for_color = tf.image.convert_image_dtype(tf.image.crop_to_bounding_box(image, 220, 260, 40, 80), tf.float32)[tf.newaxis,:,:,:]
    image_for_position_detection = tf.image.convert_image_dtype(tf.image.rgb_to_grayscale(tf.image.crop_to_bounding_box(image, 220,70,100,500)), tf.float32)[tf.newaxis,:,:,:]
    orientation=self.orientation_classifier(image_for_position_detection)[0,:]
    return tf.math.softmax(self.color_classifier(image_cropped_for_color))[0,:], orientation


model = FusedModel()

image = tf.io.decode_jpeg(tf.io.read_file('/nfs/general/shared/KeyD/1650406299896.jpg'))

model.compute(image)

2022-04-19 22:14:27.715477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 22:14:27.718876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 22:14:27.719174: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 22:14:27.719561: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.00230436, 0.9976956 ], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.68506  , 0.6585783], dtype=float32)>)

In [3]:
import glob
BASE_DIR = '/nfs/general/shared'
D_FILES = set([x.replace(BASE_DIR, '')
               for x in glob.glob('{}/KeyD/*.jpg'.format(BASE_DIR))])

In [4]:
sorted(D_FILES)

['/KeyD/1650026599127.jpg',
 '/KeyD/1650026603443.jpg',
 '/KeyD/1650026605632.jpg',
 '/KeyD/1650026607699.jpg',
 '/KeyD/1650026611962.jpg',
 '/KeyD/1650026617590.jpg',
 '/KeyD/1650026621493.jpg',
 '/KeyD/1650026623927.jpg',
 '/KeyD/1650026626966.jpg',
 '/KeyD/1650026629241.jpg',
 '/KeyD/1650026631671.jpg',
 '/KeyD/1650026633587.jpg',
 '/KeyD/1650026635605.jpg',
 '/KeyD/1650026640272.jpg',
 '/KeyD/1650026642461.jpg',
 '/KeyD/1650026655348.jpg',
 '/KeyD/1650026657506.jpg',
 '/KeyD/1650026659358.jpg',
 '/KeyD/1650026661289.jpg',
 '/KeyD/1650026665659.jpg',
 '/KeyD/1650026667845.jpg',
 '/KeyD/1650026670335.jpg',
 '/KeyD/1650026672402.jpg',
 '/KeyD/1650026674512.jpg',
 '/KeyD/1650026676580.jpg',
 '/KeyD/1650026678649.jpg',
 '/KeyD/1650026682043.jpg',
 '/KeyD/1650026686258.jpg',
 '/KeyD/1650026692955.jpg',
 '/KeyD/1650026695156.jpg',
 '/KeyD/1650026697319.jpg',
 '/KeyD/1650026702573.jpg',
 '/KeyD/1650026705344.jpg',
 '/KeyD/1650026713949.jpg',
 '/KeyD/1650026716269.jpg',
 '/KeyD/165002671865